# DATE GO
## 데이트 코스 추천
#### CBF(초반 사용자 데이터 부족시 비중 多) + CF(데이터 충분해지면 비중 多)
! 이 파일은 cafe data 기준

In [101]:
import pandas as pd
from math import log
from surprise import Reader, Dataset
from surprise import SVD, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise.dataset import DatasetAutoFolds
from collections import defaultdict
import os

# 0. Load Data
CSV 불러오기

In [2]:
# csv파일 불러오기
raw_data = pd.read_csv("./cafe_info.csv", encoding="cp949")
user_data = pd.read_csv("./user_review.csv", encoding="cp949")

mat_list = []
tag_list = ['가성비좋은',	'분위기좋은',	'감성카페',	'고급스러운',	'조용한',	'깔끔한',
            '디저트',	'인테리어',	'사진찍기좋은',	'이색적인',	'뷰가좋은',	'예쁜',	'동네핫플']

# 총 문서의 수
N = len(raw_data)

# 1. CBF
1. store_id, tags로 tf-idf 계산
    - 카테고리 별로 태그 종류가 다름.
------------------  
2. 사용자가 취향 설정시 선택한 태그들을 기반으로 **가상의 가게**를 설정
    - cosine sim은 방향이 중요하므로 cos 직전에 유사도 체크
-----------------  
3. cosine similarity matrix 구하기
    - shape : (가게수, 가게수)

--------------------------

4. 2에서 설정한 **가상의 가게**와의 유사도 + 가중치
    - 가중치 : 리뷰 개수 절대적으로 반영? minmax scaler 이용하여 상대적으로 반영?
    - 절대적 반영:
        - c : 리뷰 수, r : rating
        - ( min(c, 100) + r * 20 ) / 200
------------------  
5. sort
------------------  
6. 상위 K개 가게 추출

In [3]:
def tf(t, d):
    # d => 인덱스의 태그들, t => 태그 하나
    return d.count(t)


def idf(t):
    df = 0
    # 전체 태그들중에 해당 태그가 몇번 사용됐는디 로그로
    df += sum(raw_data[t].values)
    return log(N/(df+1))


def tfidf(t, d, i):
    # tf는 정의대로 수정
    return raw_data[t][i] * idf(t)

In [4]:
result = []

# ---------------------------------tf-----------------------------
for i in range(N):
    temp = []
    for key in tag_list:
        temp.append(raw_data[key][i])
    mat_list.append(temp)
tf_ = pd.DataFrame(mat_list, columns=tag_list)
print('tf_', tf_)

tf_      가성비좋은  분위기좋은  감성카페  고급스러운  조용한  깔끔한  디저트  인테리어  사진찍기좋은  이색적인  뷰가좋은  예쁜  \
0        1      1     0      0    0    1    0     0       1     1     0   0   
1        0      1     0      0    0    1    1     0       0     0     0   1   
2        1      1     0      0    1    1    0     0       0     0     1   1   
3        0      0     0      0    1    1    0     0       0     0     1   1   
4        1      1     0      1    1    0    1     1       0     0     1   1   
..     ...    ...   ...    ...  ...  ...  ...   ...     ...   ...   ...  ..   
859      1      1     0      1    0    0    1     1       0     0     1   1   
860      0      0     1      0    1    0    0     0       1     1     0   1   
861      1      0     0      0    1    1    0     0       0     1     0   1   
862      0      1     1      0    1    0    0     0       1     1     1   0   
863      1      0     1      0    1    1    0     1       1     0     1   0   

     동네핫플  
0       1  
1       1  
2       0  

In [5]:
# --------------------------------idf-----------------------------
result = []
for j in range(len(tag_list)):
    t = tag_list[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=tag_list, columns=["IDF"])
print('idf_', idf_)

idf_              IDF
가성비좋은   0.654550
분위기좋은   0.709484
감성카페    0.700116
고급스러운   0.726091
조용한     0.765121
깔끔한     0.690835
디저트     0.688528
인테리어    0.652325
사진찍기좋은  0.670263
이색적인    0.702450
뷰가좋은    0.654550
예쁜      0.723702
동네핫플    0.665748


In [6]:
# ------------------------------tf-idf----------------------------
result = []
for i in range(N):
    result.append([])
    d = raw_data['name'][i]
    for j in range(len(tag_list)):
        t = tag_list[j]
        result[-1].append(tfidf(t, d, i))

tfidf_ = pd.DataFrame(result, columns=tag_list)
print('tfidf_', tfidf_)

tfidf_        가성비좋은     분위기좋은      감성카페     고급스러운       조용한       깔끔한       디저트  \
0    0.65455  0.709484  0.000000  0.000000  0.000000  0.690835  0.000000   
1    0.00000  0.709484  0.000000  0.000000  0.000000  0.690835  0.688528   
2    0.65455  0.709484  0.000000  0.000000  0.765121  0.690835  0.000000   
3    0.00000  0.000000  0.000000  0.000000  0.765121  0.690835  0.000000   
4    0.65455  0.709484  0.000000  0.726091  0.765121  0.000000  0.688528   
..       ...       ...       ...       ...       ...       ...       ...   
859  0.65455  0.709484  0.000000  0.726091  0.000000  0.000000  0.688528   
860  0.00000  0.000000  0.700116  0.000000  0.765121  0.000000  0.000000   
861  0.65455  0.000000  0.000000  0.000000  0.765121  0.690835  0.000000   
862  0.00000  0.709484  0.700116  0.000000  0.765121  0.000000  0.000000   
863  0.65455  0.000000  0.700116  0.000000  0.765121  0.690835  0.000000   

         인테리어    사진찍기좋은     이색적인     뷰가좋은        예쁜      동네핫플  
0    0.000000  0

In [44]:
def func(idx):
#     if raw_data['count'][idx] > 100 : c = 100
#     else : c = raw_data['count'][idx]
    return (min(100, raw_data['count'][idx]) + raw_data['rating'][idx]*20)/200

In [8]:
def compute_cos_sim(arr): #cosine 유사도
    cosine_matrix = cosine_similarity(arr, arr)
    spotId = {}
    for idx, c in enumerate(raw_data['name']):  spotId[idx] = c
    id2spot = {}
    for idx, c in spotId.items() : id2spot[c] = idx
    return cosine_matrix, spotId, id2spot

In [9]:
def get_k_sim(name, k): #가중치 계산
    idx = id2spot[name]
    sim_scores = [(i, c + func(i) ) for i, c in enumerate(cosine_matrix[idx]) if i != idx]
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)
    sim_scores = [(spotId[i], score) for i, score in sim_scores[0:k]]
    return sim_scores

def get_k_sim2(name, k): #가중치 없이 유사도만
    idx = id2spot[name]
    sim_scores = [(i, c ) for i, c in enumerate(cosine_matrix[idx]) if i != idx]
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)
    sim_scores = [(spotId[i], score) for i, score in sim_scores[0:k]]
    return sim_scores

# 2. CF
###  KNNBasic (K-nearest neighbor) in surprise  
1. user_id, store_id, rating 사용. KNN 알고리즘 기반으로 **'나'와 유사한 유저 K명** 추출
    - '나'와의 유사성은 지금까지의 '나'의 리뷰를 기반으로.
    - '나'의 리뷰가 있어야 함.
    - 없을 때에는 CBF 100% 반영
---------------------
2. K명의 유저들의 최근 리뷰 내역 참고하여 5점을 준 가게들 리스트화
    - 평균적으로 평점을 높게 주는 사람들 vs 꼼꼼하게 체크하고 정말 만족해야 5점을 주는 사람들 (?Baseline)
    - 위의 경우 5점의 가치가 같은가?
------------------
3. sort 우선순위 : 
    - 방문횟수가 많을수록 - 전체 유저(count)
    - 평점이 높을수록 - 전체 유저
    - (인기 많은 가게 추천)
-----------------

In [31]:
reader = Reader(rating_scale=(1, 5.0))
data = Dataset.load_from_df(user_data[['user_id', 'store_id', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=.25, random_state=1234)

In [64]:
# 수행시마다 동일한 결과 도출을 위해 random_state 설정
algo = SVD(n_factors=100, random_state=1234)

    
# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.7381


0.7380841618011615

In [65]:
print('prediction type : ', type(predictions), 'size : ', len(predictions))
print('prediction 결과')
predictions[:]

prediction type :  <class 'list'> size :  10
prediction 결과


[Prediction(uid=4, iid=2108, r_ui=3.0, est=4.011070955107169, details={'was_impossible': False}),
 Prediction(uid=2, iid=2616, r_ui=4.0, est=4.514326545786372, details={'was_impossible': False}),
 Prediction(uid=4, iid=2090, r_ui=5.0, est=4.011070955107169, details={'was_impossible': False}),
 Prediction(uid=3, iid=2040, r_ui=4.0, est=4.21879464626292, details={'was_impossible': False}),
 Prediction(uid=3, iid=2016, r_ui=5.0, est=4.21879464626292, details={'was_impossible': False}),
 Prediction(uid=2, iid=2034, r_ui=5.0, est=4.411451074399704, details={'was_impossible': False}),
 Prediction(uid=3, iid=2020, r_ui=3.0, est=4.21879464626292, details={'was_impossible': False}),
 Prediction(uid=2, iid=2741, r_ui=5.0, est=4.411451074399704, details={'was_impossible': False}),
 Prediction(uid=4, iid=2366, r_ui=4.0, est=4.011070955107169, details={'was_impossible': False}),
 Prediction(uid=2, iid=2664, r_ui=5.0, est=4.411451074399704, details={'was_impossible': False})]

In [98]:
uid = str(4)
iid = str(2108)
pred = algo.predict(uid, iid)
print(pred)

user: 4          item: 2108       r_ui = 5.00   est = 4.27   {'was_impossible': False}


In [68]:
opts = {'name' : 'pearson', 'user_based':True}
algo2 = KNNBasic(sim_option=opts)

algo2.fit(trainset)
predictions2 = algo2.test(testset)
accuracy.rmse(predictions2)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8390


0.8390470785361213

In [69]:
print('prediction type : ', type(predictions2), 'size : ', len(predictions2))
print('prediction 결과')
predictions2[:]

prediction type :  <class 'list'> size :  10
prediction 결과


[Prediction(uid=4, iid=2108, r_ui=3.0, est=4.266666666666667, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=2, iid=2616, r_ui=4.0, est=5.0, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=4, iid=2090, r_ui=5.0, est=4.266666666666667, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=3, iid=2040, r_ui=4.0, est=4.266666666666667, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=3, iid=2016, r_ui=5.0, est=4.266666666666667, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=2, iid=2034, r_ui=5.0, est=4.266666666666667, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=3, iid=2020, r_ui=3.0, est=4.266666666666667, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=2, iid=2741, r_ui=5.0, est=4.266666666666667, detai

In [100]:
# uid = str(2)
# iid = str(2616)
pred = algo2.test([[2,2616,4.0]]) # uid, iid, r_ui(None) --> 리스트 형태로!
print(pred)

[Prediction(uid=2, iid=2616, r_ui=4.0, est=5.0, details={'actual_k': 1, 'was_impossible': False})]


In [102]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.
    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [105]:
top_n = get_top_n(predictions, n=1)

In [106]:
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

4 [2108]
2 [2616]
3 [2040]


In [80]:
# Let's build a pandas dataframe with all the predictions

def get_Iu(uid):
    """Return the number of items rated by given user
    
    Args:
        uid: The raw id of the user.
    Returns:
        The number of items rated by the user.
    """
    
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError:  # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """Return the number of users that have rated given item
    
    Args:
        iid: The raw id of the item.
    Returns:
        The number of users that have rated the item.
    """
    
    try:
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:  # item was not part of the trainset
        return 0

dfid = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])    
dfid['Iu'] = dfid.uid.apply(get_Iu)
dfid['Ui'] = dfid.iid.apply(get_Ui)
dfid['err'] = abs(dfid.est - dfid.rui)

In [110]:
dfid.sort_values(by='est', ascending=False)[:10]

,uid,iid,rui,est,details,Iu,Ui,err
1,2,2616,4.0,4.514327,{'was_impossible': False},6,1,0.514327
5,2,2034,5.0,4.411451,{'was_impossible': False},6,0,0.588549
7,2,2741,5.0,4.411451,{'was_impossible': False},6,0,0.588549
9,2,2664,5.0,4.411451,{'was_impossible': False},6,0,0.588549
3,3,2040,4.0,4.218795,{'was_impossible': False},7,0,0.218795
4,3,2016,5.0,4.218795,{'was_impossible': False},7,0,0.781205
6,3,2020,3.0,4.218795,{'was_impossible': False},7,0,1.218795
0,4,2108,3.0,4.011071,{'was_impossible': False},7,0,1.011071
2,4,2090,5.0,4.011071,{'was_impossible': False},7,0,0.988929
8,4,2366,4.0,4.011071,{'was_impossible': False},7,0,0.011071


In [108]:
best_predictions = dfid.sort_values(by='err')[:10]
best_predictions

,uid,iid,rui,est,details,Iu,Ui,err
8,4,2366,4.0,4.011071,{'was_impossible': False},7,0,0.011071
3,3,2040,4.0,4.218795,{'was_impossible': False},7,0,0.218795
1,2,2616,4.0,4.514327,{'was_impossible': False},6,1,0.514327
5,2,2034,5.0,4.411451,{'was_impossible': False},6,0,0.588549
7,2,2741,5.0,4.411451,{'was_impossible': False},6,0,0.588549
9,2,2664,5.0,4.411451,{'was_impossible': False},6,0,0.588549
4,3,2016,5.0,4.218795,{'was_impossible': False},7,0,0.781205
2,4,2090,5.0,4.011071,{'was_impossible': False},7,0,0.988929
0,4,2108,3.0,4.011071,{'was_impossible': False},7,0,1.011071
6,3,2020,3.0,4.218795,{'was_impossible': False},7,0,1.218795


In [91]:
worst_predictions = dfid.sort_values(by='err')[-5:]
worst_predictions

,uid,iid,rui,est,details,Iu,Ui,err
9,2,2664,5.0,4.411451,{'was_impossible': False},6,0,0.588549
4,3,2016,5.0,4.218795,{'was_impossible': False},7,0,0.781205
2,4,2090,5.0,4.011071,{'was_impossible': False},7,0,0.988929
0,4,2108,3.0,4.011071,{'was_impossible': False},7,0,1.011071
6,3,2020,3.0,4.218795,{'was_impossible': False},7,0,1.218795


In [16]:
def recur_dictify(frame):
    if len(frame.columns) == 1:
        if frame.values.size == 1: return frame.values[0][0]
        return frame.values.squeeze()
    grouped = frame.groupby(frame.columns[0])
    d = {k: recur_dictify(g.iloc[:, 1:]) for k, g in grouped}
    return d

In [17]:
df_to_dict = recur_dictify(user_data[['user_id', 'store_id', 'rating']])

In [18]:
user_list = []
store_set = set()

for user in df_to_dict:
    user_list.append(user)

    for store in df_to_dict[user]:
        store_set.add(store)

store_list = list(store_set)

In [19]:
# 학습할 데이터를 준비한다.
rating_dic = {
        'user_id': [],
        'store_id': [],
        'rating': []
    }

In [20]:
# 유저 수 만큼 반복
for user in df_to_dict:
	# 해당 유저의 리뷰 수 만큼 반복
    for store in df_to_dict[user]:
        u_index = user_list.index(user)
        s_index = store_list.index(store)
        rating = df_to_dict[user][store]

        rating_dic['user_id'].append(u_index)
        rating_dic['store_id'].append(s_index)
        rating_dic['rating'].append(rating)
        
# 데이터셋 만들기
df = pd.DataFrame(rating_dic)

In [21]:
# # 데이터를 읽어들이는 객체 생성 (rating scales : 평점 범위)
# reader = .Reader(rating_scale=(1.0, 5.0))

col_list = ['user_id', 'store_id', 'rating']
# data = surprise.Dataset.load_from_df(user_data[col_list], reader)

# 학습
trainset = data.build_full_trainset()

option={'name':'pearson', 'user_based':True}
algo = KNNBasic(sim_options=option)
algo.fit(trainset)


Computing the pearson similarity matrix...
Done computing similarity matrix.


In [22]:
index = user_list.index(1)#1번 사람 : 0번 인덱스
result = algo.get_neighbors(index, k=1)
print(result)

[2]


In [23]:
recommand_dic = {
        'user_id': [],
        'store_id': [],
    }

for user_key in user_data['user_id'].unique():
    index = user_list.index(user_key)
    result = algo.get_neighbors(index, k=1)
    recom_set = set()
    for i in result:
            max_rating = data.df[data.df['user_id'] == user_list[i]]['rating'].max()
            recom_stores = data.df[(data.df['rating'] == max_rating) & (data.df['user_id'] == user_list[i])][
                'store_id'].values
            for item in recom_stores:
                recom_set.add(item)

            for item in recom_set:
                recommand_dic['user_id'].append(user_key)
                recommand_dic['store_id'].append(item)

pickle = pd.DataFrame(recommand_dic)

# BASE_DIR = os.path.dirname(os.path.abspath(__file__))
# path = os.path.join(BASE_DIR, 'KNN.p')
path = "./KNN.p"
pd.to_pickle(pickle, path)

In [41]:
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}
gs = GridSearchCV(SVD, param_grid, measures=['RMSE'], cv = 3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8366391943369434
{'n_epochs': 40, 'n_factors': 100}


In [43]:
param_grid = {}
gs2 = GridSearchCV(KNNBasic, param_grid, measures=['RMSE'], cv = 3)
gs2.fit(data)

print(gs2.best_score['rmse'])
print(gs2.best_params['rmse'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
0.9449899809824899
{}
